In [34]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pandas as pd

from datetime import datetime,timedelta

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['measurements', 'stations']

In [8]:
Measurements = Base.classes.measurements
Stations = Base.classes.stations

In [9]:
session = Session(engine)

In [28]:
current_time = datetime.now()

past_year = current_time - timedelta(days=365)

measurements_year = session.query(Measurements.date,Measurements.prcp,Measurements.tobs).filter(
    Measurements.date > past_year).all()

In [30]:
measure_records = []
for measure in measurements_year:
    measure_records.append(measure._asdict())

In [35]:
measurements_df = pd.DataFrame.from_records(measure_records)
measurements_df.head()

,date,prcp,tobs
0,2016-12-23,0.01,72
1,2016-12-24,0.01,74
2,2016-12-25,0.00,74
3,2016-12-26,0.02,74
4,2016-12-27,0.00,74
